## Preparation

In [1]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [2]:
%pip install bertopic

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
# General imports
import pandas as pd
from collections import defaultdict

# NLP imports
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from bertopic import BERTopic

# Google imports
from google.colab import drive

## Load the dataset

In [4]:
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [5]:
df = pd.read_pickle('gdrive/MyDrive/mathesis/df_ml_ready_2010.pkl')
df

,window_id,paragraph_id,item_of_business,person_id,first_name,last_name,council,party,in_admin_role,text,date,session_title,session_id,tokens,sentences_window
0,0,325493,1. Mitteilungen,8294,Monika,Spring,Kantonsrat,SP,False,"– KR-Nr. 317/2009, Verhältnis von Einnahmen un...",2010-01-04,147. Ratssitzung,1180,"[–, KR-Nr, ., 317/2009, ,, Verhältnis, von, Ei...","– KR-Nr. 317/2009, Verhältnis von Einnahmen un..."
1,1,325493,1. Mitteilungen,8294,Monika,Spring,Kantonsrat,SP,False,"– KR-Nr. 317/2009, Verhältnis von Einnahmen un...",2010-01-04,147. Ratssitzung,1180,"[–, KR-Nr, ., 317/2009, ,, Verhältnis, von, Ei...","317/2009, Verhältnis von Einnahmen und Ausgabe..."
2,2,325493,1. Mitteilungen,8294,Monika,Spring,Kantonsrat,SP,False,"– KR-Nr. 317/2009, Verhältnis von Einnahmen un...",2010-01-04,147. Ratssitzung,1180,"[–, KR-Nr, ., 317/2009, ,, Verhältnis, von, Ei...","308/2009, ZVV-Pannen – Sorgen um die Qualität ..."
3,3,325493,1. Mitteilungen,8294,Monika,Spring,Kantonsrat,SP,False,"– KR-Nr. 317/2009, Verhältnis von Einnahmen un...",2010-01-04,147. Ratssitzung,1180,"[–, KR-Nr, ., 317/2009, ,, Verhältnis, von, Ei...","Sabine Ziegler (SP, Zürich) – KR-Nr. 309/2009..."
4,4,325493,1. Mitteilungen,8294,Monika,Spring,Kantonsrat,SP,False,"– KR-Nr. 317/2009, Verhältnis von Einnahmen un...",2010-01-04,147. Ratssitzung,1180,"[–, KR-Nr, ., 317/2009, ,, Verhältnis, von, Ei...","309/2009, Dmitri Medwedew Yves Senn (SVP, Win..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
674064,674064,348638,7. Verschiedenes,21542,Sylvie,Matter,Kantonsrat,SP,False,Ich muss Ihnen an dieser Stelle einen weiteren...,2023-04-17,226. Ratssitzung,2734,"[Ich, muss, Ihnen, an, dieser, Stelle, einen, ...",In ihrer Amtszeit wurde danach das patriarchal...
674065,674065,348638,7. Verschiedenes,21542,Sylvie,Matter,Kantonsrat,SP,False,Ich muss Ihnen an dieser Stelle einen weiteren...,2023-04-17,226. Ratssitzung,2734,"[Ich, muss, Ihnen, an, dieser, Stelle, einen, ...",Kopps Leistungen wurden später überschattet vo...
674066,674066,348638,7. Verschiedenes,21542,Sylvie,Matter,Kantonsrat,SP,False,Ich muss Ihnen an dieser Stelle einen weiteren...,2023-04-17,226. Ratssitzung,2734,"[Ich, muss, Ihnen, an, dieser, Stelle, einen, ...","Es dauerte lange, bis die Affäre in den Hinter..."
674067,674067,348638,7. Verschiedenes,21542,Sylvie,Matter,Kantonsrat,SP,False,Ich muss Ihnen an dieser Stelle einen weiteren...,2023-04-17,226. Ratssitzung,2734,"[Ich, muss, Ihnen, an, dieser, Stelle, einen, ...",Sie selber zog sich länger aus der Öffentlichk...


## Prepare stopwords

In [6]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [7]:
stop_words = stopwords.words('german')
len(stop_words)

232

In [8]:
paragraph_tokens = df.groupby('paragraph_id')['tokens'].first()

In [9]:
frequency = defaultdict(int)
for tokens in list(paragraph_tokens):
    for token in set(tokens):
        frequency[token.lower()] += 1

In [10]:
for token, f in frequency.items():
  if (f/len(paragraph_tokens) > 0.03 or
      len(token) <= 2 or
      sum([c.isalpha() for c in token]) / len(token) < 0.6):
    stop_words.append(token)

In [11]:
len(stop_words)

17998

## Prepare topic model

In [12]:
vectorizer_model = CountVectorizer(stop_words=stop_words)
model = BERTopic(verbose=True, 
                 language="german", 
                 n_gram_range=(1, 3), 
                 vectorizer_model=vectorizer_model, 
                 nr_topics='auto', 
                 min_topic_size=45, 
                 top_n_words=8) 

# Fit topic model

In [13]:
docs = list(df['sentences_window'])

In [14]:
topics, _ = model.fit_transform(docs)

Batches:   0%|          | 0/21065 [00:00<?, ?it/s]

2023-04-30 21:10:06,596 - BERTopic - Transformed documents to Embeddings
2023-04-30 21:23:43,538 - BERTopic - Reduced dimensionality
2023-04-30 21:26:03,706 - BERTopic - Clustered reduced embeddings
2023-04-30 21:28:04,394 - BERTopic - Reduced number of topics from 564 to 290


# Store model

In [15]:
model.save("gdrive/MyDrive/mathesis/topic_model_bertopic_2010")

/usr/local/lib/python3.10/dist-packages/scipy/sparse/_index.py:146: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


# Calculate Coherence Score

In [ ]:
type(topics)